In [12]:
# Работа с сервером
import requests
from pprint import pprint
import json

# Распаковка и парсинг
import zipfile
import pandas as pd
import io
from bs4 import BeautifulSoup
from datetime import datetime


from urllib.parse import urljoin, urlparse
import os
import shutil
import base64
import importlib
import re

from pathlib import Path

In [13]:
# Загружаем словари для отправки Мain info
from functions import get_dict

ranks = get_dict('ranks')
gender = get_dict('gender')
marital_statuses = get_dict('marital_statuses')
resident_statuses = get_dict('resident_statuses')
languages = get_dict('languages')


In [14]:
PATH = 'out/out_min'

In [15]:
# Все HTML файлы из папки
html_files = [str(p) for p in Path(PATH).rglob('*.html')]


In [52]:
import functions
importlib.reload(functions)
# обрабатываем все файлы из папки
for file in html_files:
    # получаем объект soup
    soup = functions.get_html_content(file)
    # парсим всю инфу
    main_info = functions.main_parser(soup)
    # и заметки
    notes = functions.parse_notes(soup)
    # собираем краткую инфу
    short_keys = [
        #  'Name / Surname:',
        # 'Position applied for:',
        #'Birthday / Place of birth:',
        # 'Country of residence / City:',
        # 'Citizenship:',
        # 'Phones:',
        'E-mail:'
        ]
    short_info = {k: main_info['Main info'][k] for k in short_keys}
    # # смотрим последний рейс
    # last_voyage = main_info['Sea service (last 5 years)'][0] if main_info['Sea service (last 5 years)'] else None
    # is_photo = f'Фото {"Есть" if functions.get_photo_simple(soup) else "Нет"}'
    # вывод резюме
    # pprint(file)
    # print(is_photo)
    pprint(short_info, sort_dicts=False)
    # pprint(last_voyage)
    # pprint(notes)
    # print()
    

{'E-mail:': 's-semenenko@bk.ru'}
{'E-mail:': 'cap.omarmohamed@gmail.com'}
{'E-mail:': '* * * * * *'}
{'E-mail:': 'and1979@inbox.lv'}
{'E-mail:': 'iazvinskii.maksim@icloud.com'}
{'E-mail:': 'mykhailoianishevskyi@gmail.com'}
{'E-mail:': 'alex.liakhovetskyi@gmail.com'}


In [ ]:
# обработка спарсенных данных и получение необходимой информации
from functions import get_names, extract_date_to_iso, get_emails_list

#Обработка ФИО
name, middle_name, surname = get_names(main_info['Main info']['Name / Surname:'])

#Обработка даты и места рождения
date_of_birth, place_of_birth = extract_date_to_iso(main_info['Main info']['Birthday / Place of birth:'])

# Добавление емейлов
emails = get_emails_list(main_info['Main info']['E-mail:'])

In [ ]:
# проверка работоспособности 
# Будет удалено

import functions
importlib.reload(functions)

for file in html_files:
    # получаем объект soup
    soup = functions.get_html_content(file)
    # парсим всю инфу
    main_info = functions.main_parser(soup)
    print(main_info['Main info']['E-mail:'])
    print(functions.get_emails_list(main_info['Main info']['E-mail:']))
    print()

s-semenenko@bk.ru
[{'email': 's-semenenko@bk.ru', 'comment': 'comment', 'uuid': None}]

cap.omarmohamed@gmail.com
[{'email': 'cap.omarmohamed@gmail.com', 'comment': 'comment', 'uuid': None}]

* * * * * *
None

and1979@inbox.lv
[{'email': 'and1979@inbox.lv', 'comment': 'comment', 'uuid': None}]

iazvinskii.maksim@icloud.com
[{'email': 'iazvinskii.maksim@icloud.com', 'comment': 'comment', 'uuid': None}]

mykhailoianishevskyi@gmail.com
[{'email': 'mykhailoianishevskyi@gmail.com', 'comment': 'comment', 'uuid': None}]

alex.liakhovetskyi@gmail.com
[{'email': 'alex.liakhovetskyi@gmail.com', 'comment': 'comment', 'uuid': None}]



In [ ]:
# POST /seafarers/main
 seafarer_dict = {
     # "photo": FILE, 
     "name": name,
     "middle_name": middle_name,
     "surname": surname,
     
    
   
     # "rank_id": INTEGER,
     # "additional_ranks_id": [int, int, int, ..],
     
     # "calling_name": STRING,
     # "available_from": DATE,
     # "available_to": DATE,
     
     "date_of_birth": date_of_birth,
     "place_of_birth": place_of_birth,
     
     
     # "gender_id": INTEGER,
     
     # "marital_status_id": INTEGER,
     # "nationality_country_id": INTEGER,
    
     "emails": emails
     
     # "resident_status_id": INTEGER,
     # "fast_note": STRING,
     # "phone_numbers": [
     #     {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
     # ],
     # "personal_id": INTEGER,
     # "language_id": INTEGER,
 }
 seafarer_dict

In [ ]:

# # ячейка для присвоения значений переменным
# photo = 
# name =
# middle_name =
# surname =

# rank_id = get_id(ranks,extract_anything('out\\out_min\\seafarers_104588.html', key))

# additional_ranks_id =
# calling_name =
# available_from =
# available_to =
# # date_of_birth =
# place_of_birth =
# gender_id =
# marital_status_id =
# nationality_country_id =
# emails =
# resident_status_id =
# fast_note =
# phone_numbers =
# personal_id =
# language_id =

# # Словарь формат json с основной информацией о моряке,
# # который будет отправлен для получения uuid моряка
# # - по нему в дальнейшем будет добавляться информация в БД
# main = {
#     "photo": photo #FILE, ?? 
    
#     "name": name #STRING,
#     "middle_name": middle_name #STRING,
#     "surname": surname #STRING,
#     "rank_id": rank_id #INTEGER,
#     "additional_ranks_id": additional_ranks_id #[int, int, int, ..],
#     "calling_name": calling_name #STRING,
#     "available_from": available_from #DATE,
#     "available_to": available_to #DATE,
#     "date_of_birth": date_of_birth #DATE,
#     "place_of_birth": place_of_birth #STRING,
#     "gender_id":gender_id # INTEGER,
#     "marital_status_id":marital_status_id # INTEGER,
#     "nationality_country_id":nationality_country_id # INTEGER,
#     "emails":emails # [
#         # {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     # ],
#     "resident_status_id": resident_status_id #INTEGER,
#     "fast_note": fast_note #STRING,  
#     "phone_numbers": phone_numbers #[
#         # {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     # ],
#     "personal_id": personal_id #INTEGER,
#     "language_id": language_id #INTEGER,
# }

In [ ]:
# # определение url ссылок для использования

# # основной домен
# domain = 'https://staffdev.crew-man.com/api/v1/'

# # Добавление основной инфы моряка:POST
# main_info = 'seafarers/main'

# # seafarer_uuid = response_main.text
# # Добавление адресов моряка:POST
# # adress = f'seafarers/{seafarer_uuid}/addresses'

# # Добавление сертификатов (документов) моряка:POST
# # certificates = f'seafarers/{seafarer_uuid}/certificates'

# # Добавление родственников (kins) моряка:POST
# # relatives = f'seafarers/{seafarer_uuid}/relatives'

# # Добавление sea service моряка:POST
# contracts =  'contracts'

# route = main_info
# url = domain + route


In [ ]:
# from bs4 import MarkupResemblesLocatorWarning
# import warnings
# warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [ ]:
# post ={
#     "photo": FILE, ??
#     "name": NAME,
#     "middle_name": STRING,
#     "surname": SURNAME,
#     "rank_id": INTEGER,
#     # "additional_ranks_id": [int, int, int, ..],
#     "calling_name": STRING,
#     "available_from": DATE,
#     "available_to": DATE,
#     "date_of_birth": DATE_OF_BIRTH,
#     "place_of_birth": STRING,
#     "gender_id": INTEGER,
#     "marital_status_id": INTEGER,
#     "nationality_country_id": INTEGER,
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER,
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER,
#     "language_id": INTEGER,
# }

In [ ]:
# main = {
    
#     "name": [name.strip() for name in 'Name / Surname:'.split('/')][0],
    
#     "surname": [name.strip() for name in 'Name / Surname:'.split('/')][1][:-1],
    
#     "rank_id": INTEGER, я так понимаю это ид должностей где они беруться?
#     "additional_ranks_id": [int, int, int, ..],
    
    
#     "date_of_birth": DATE,
#     # "place_of_birth": STRING,
    
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER, резидент РФ?
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER, где берется?
#     "language_id": INTEGER, где берется?
# }